# Simulación DES

## Simulación basada en eventos.

Problema: Simule las operaciones de un taller de autos que cuenta con una estación para reparar los vehiculos. Se sabe que el tiempo promedio entre llegadas es de 10 minutos segun un proceso Poisson. Debido a la capacidad del taller, los autos pueden esperar en una cola para ser atendidos, sin embargo por temas estructurales solo puede existir un máximo de 10 autos en espera, y los siguientes autos que lleguen abandonaran el taller en busca de uno que los atienda. el tiempo de reparación del taller resulta ser uniforme entre 10 a 15 minutos una vez ingresa el vehiculo a la estación.
Indique la cantidad de autos que ingresan al taller, los autos que abandonan el taller luego de ser reparados y los autos que abandonan el taller por que la capacidad de la cola se ve superada. 

Simule las operaciones de un día para una joranada laboral de 8 horas considerando que se empieza a las 9:00 am

Librerías recomendadas:
* **datetime**
* **random**
* **collections**

¿Qué son las properties?

https://www.quora.com/Python-newbie-what-are-attributes-and-properties-in-a-class


In [1]:
from collections import deque
from random import expovariate, randint, uniform, seed
from datetime import datetime, timedelta
import time
seed(1)

In [2]:
class Auto:
    _id = 0
    def __init__(self, tiempo_instancia):
        Auto._id += 1
        self.id = Auto._id
        self.tiempo_llegada = tiempo_instancia
        self.estacion = None
        # self.tiempo_abandono_cola = None
        self.tiempo_abandono_taller = None


    def generar_tiempo_abandono_taller(self, tiempo_actual):
        self.tiempo_abandono_taller = tiempo_actual + timedelta(minutes=int(uniform(10,15)))

    def generar_tiempo_total_sistema(self, tiempo_actual):
        self.tiempo_total_sistema = tiempo_actual - self.tiempo_llegada
        print(self.tiempo_total_sistema)

    def __str__(self):
        return "-> {}".format(self.id)

In [3]:
# Clase principal que contiene el motor de la simulacion
class Taller:
    def __init__(self, tiempo_simulacion, tasa_llegada, capacidad):
        # usamos datetime para manejar temporalidad
        date = datetime(2019, 8, 1)
        newdate = date.replace(hour=9)

        self.tiempo_actual = newdate
        # self.tiempo_actual = 0
        self.tiempo_maximo = newdate + timedelta(hours = tiempo_simulacion)

        self.tasa_llegada = tasa_llegada
        self.estacion = {"E1" : None}
        self.proximo_auto_llega = self.tiempo_actual + timedelta(minutes=int(expovariate(1/tasa_llegada)))
        # self.proximo_auto_termina = None
        self.capacidad_cola = capacidad
        self.cola = deque()

        # las variables para el calculo de estadisticas se dejan en el contructor
        self.cantidad_autos = 0 # son los autos que llegan
        self.cantidad_autos_perdidos = 0
        self.abandonos = 0

        # manejamos una lista con todos los tiempos de abandono que se generan
        self.tiempos_abandono_taller = [[self.tiempo_actual.replace(year=3000), None]]
        # self.tiempos_abandono_autos = [[[float("inf"), None]]]


    # usamos properties para trabajar con mayor comodidad el atributo del proximo auto que termina de ser atendido
    @property
    def proximo_auto_termina(self):
        # Esta la próxima persona que terminará de ser atendida con su tiempo asociado
        x, y = self.tiempos_abandono_taller[0]
        return x, y

    @property
    def proximo_evento(self):
        tiempos = [self.proximo_auto_llega,
                   self.proximo_auto_termina[0]]
        # print(tiempos)
        tiempo_prox_evento = min(tiempos)

        if tiempo_prox_evento >= self.tiempo_maximo:
            return "fin"
        # print(tiempo_prox_evento)
        eventos = ["llegada_auto", "abandono_taller"]
        return eventos[tiempos.index(tiempo_prox_evento)]

    # funcion que define la llegada de autos al taller
    def llegar_auto(self):
        time.sleep(0.4)
        self.tiempo_actual = self.proximo_auto_llega
        self.proximo_auto_llega = self.tiempo_actual + timedelta(minutes=int(expovariate(1/self.tasa_llegada)))
        auto = Auto(self.tiempo_actual)
        print("\r\r\033[91m[LLEGADA]\033[0m ha llegado un auto id: {} {}".format(auto._id,self.tiempo_actual))

        if len(self.cola) == self.capacidad_cola:
            print("[COLA LLENA!!!] Se ha llenado la cola de espera para el taller")
            self.cantidad_autos += 1
            self.cantidad_autos_perdidos += 1
        elif self.estacion["E1"] == None:
            #self.tiempo_sistema_vacio += (self.tiempo_actual - self.ultimo_tiempo_actual_vacio)
            self.estacion["E1"] = auto
            self.estacion["E1"].estacion = "E1"
            self.estacion["E1"].generar_tiempo_abandono_taller(self.tiempo_actual)
            self.tiempos_abandono_taller.append((auto.tiempo_abandono_taller, auto))
            self.tiempos_abandono_taller.sort(key=lambda z: datetime.strftime(z[0], "%Y-%m-%d-%H-%M"))
            print("\r\r\033[92m[INGRESO ESTACION]\033[0m ha ingresado un auto a E1 id: {} {}".format(
                self.estacion["E1"]._id, self.tiempo_actual))
            self.cantidad_autos += 1
        else:
            self.cola.append(auto)
        #print(self.estacion["E1"])

    # funcion que defin la salida de autos del taller
    def abandono_taller(self):
        time.sleep(0.4)
        #print("quien va a abandonar "+ str(self.proximo_auto_termina[1]))
        self.tiempo_actual, auto_sale = self.proximo_auto_termina
        #print("quien va a abandonar " + str(auto_sale))
        if len(self.cola) > 0: # A tiene prioridad sobre B
            # Si hay, la proxima persona pasa
            print('[RETIRA] Se ha desocupado la Estacion, abandona el auto id {} {}'.
                  format(auto_sale, self.tiempo_actual))
            prox_auto = self.cola.popleft()
            #print(prox_auto)
            prox_auto.estacion = auto_sale.estacion
            self.auto_pasa_a_ser_atendido(prox_auto, auto_sale.estacion)
        else:
            print("[RETIRA] La estacion termina de atender al auto id: {}, esta desocupado pero "
                  "no hay autos en cola {}".format(auto_sale._id, self.tiempo_actual))
            self.estacion[auto_sale.estacion] = None

        self.tiempos_abandono_taller.pop(0)
        self.abandonos += 1


    # funcion que apoya el ingreso de autos 
    def auto_pasa_a_ser_atendido(self, auto, e):
        time.sleep(0.4)
        self.estacion[e] = auto
        self.estacion[e].generar_tiempo_abandono_taller(self.tiempo_actual)
        self.tiempos_abandono_taller.append((self.estacion[e].tiempo_abandono_taller, auto))
        self.tiempos_abandono_taller.sort(key=lambda z: datetime.strftime(z[0], "%Y-%m-%d-%H-%M"))
        print("\r\r\033[92m[INGRESO ESTACION]\033[0m ha ingresado un auto a E1 id: {0} {1}".format(auto,self.tiempo_actual))
        self.cantidad_autos += 1


    # motor de la simulacion 
    def run(self):
        while self.tiempo_actual < self.tiempo_maximo:
            evento = self.proximo_evento
            # print("en el modulo hay {}".format(self.modulo_atencion))
            if evento == "fin":
                self.tiempo_actual = self.tiempo_maximo
                break
            elif evento == "llegada_auto":
                self.llegar_auto()
                 # hasta aqui voy bien
            elif evento == "abandono_taller":
                self.abandono_taller()
                
    def show(self):
        print("la cantidad de autos que llegaron al taller {}".format(self.cantidad_autos))
        print("la cantidad de autos que se aburrieron de esperar {}".format(self.cantidad_autos_perdidos))
        print("la cantidad de autos que abandondad el taller {}".format(self.abandonos))
        

In [4]:
new_taller = Taller(8,10,10)
new_taller.run()


[LLEGADA] ha llegado un auto id: 1 2019-08-01 09:01:00
[INGRESO ESTACION] ha ingresado un auto a E1 id: 1 2019-08-01 09:01:00
[RETIRA] La estacion termina de atender al auto id: 1, esta desocupado pero no hay autos en cola 2019-08-01 09:14:00
[LLEGADA] ha llegado un auto id: 2 2019-08-01 09:19:00
[INGRESO ESTACION] ha ingresado un auto a E1 id: 2 2019-08-01 09:19:00
[LLEGADA] ha llegado un auto id: 3 2019-08-01 09:21:00
[LLEGADA] ha llegado un auto id: 4 2019-08-01 09:26:00
[RETIRA] Se ha desocupado la Estacion, abandona el auto id -> 2 2019-08-01 09:31:00
[INGRESO ESTACION] ha ingresado un auto a E1 id: -> 3 2019-08-01 09:31:00
[LLEGADA] ha llegado un auto id: 5 2019-08-01 09:36:00
[LLEGADA] ha llegado un auto id: 6 2019-08-01 09:36:00
[LLEGADA] ha llegado un auto id: 7 2019-08-01 09:36:00
[RETIRA] Se ha desocupado la Estacion, abandona el auto id -> 3 2019-08-01 09:44:00
[INGRESO ESTACION] ha ingresado un auto a E1 id: -> 4 2019-08-01 09:44:00
[LLEGADA] ha llegado un auto id: 8 2019-

KeyboardInterrupt: 